In [69]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from collections import deque
from sklearn import preprocessing
from random import shuffle

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , LSTM , GRU , BatchNormalization 

In [2]:
main_dataframe = pd.DataFrame()

In [3]:
currencies = ['BCH-USD', 'BTC-USD', 'ETH-USD', 'LTC-USD']
for c in currencies:
    dataset = '../RNN/predictFinance/'+c+'.csv'
    df = pd.read_csv(dataset ,  names=['time', 'low', 'high', 'open', 'close', 'volume'])
    df.rename(columns={
        'close':c+'_close',
        'volume':c+'_volume'
    },
    inplace=True)
    df.set_index('time' , inplace=True)
    df = df[[c + '_close', c + '_volume']]
    if len(main_dataframe) ==0 : 
        main_dataframe = df 
    else:
        main_dataframe = main_dataframe.join(df)

In [4]:
main_dataframe.head()

,BCH-USD_close,BCH-USD_volume,BTC-USD_close,BTC-USD_volume,ETH-USD_close,ETH-USD_volume,LTC-USD_close,LTC-USD_volume
time,,,,,,,,
1528968660,871.719971,5.675361,6489.549805,0.587100,NaN,NaN,96.580002,9.647200
1528968720,870.859985,26.856577,6487.379883,7.706374,486.01001,26.019083,96.660004,314.387024
1528968780,870.099976,1.124300,6479.410156,3.088252,486.00000,8.449400,96.570000,77.129799
1528968840,870.789978,1.749862,6479.410156,1.404100,485.75000,26.994646,96.500000,7.216067
1528968900,870.000000,1.680500,6479.979980,0.753000,486.00000,77.355759,96.389999,524.539978


In [5]:
main_dataframe.fillna(method='ffill' , inplace=True)

In [6]:
main_dataframe.fillna(method='bfill' , inplace=True)

In [7]:
main_dataframe.isna().sum()

BCH-USD_close     0
BCH-USD_volume    0
BTC-USD_close     0
BTC-USD_volume    0
ETH-USD_close     0
ETH-USD_volume    0
LTC-USD_close     0
LTC-USD_volume    0
dtype: int64

In [8]:
main_dataframe['feature'] = main_dataframe['LTC-USD_close'].shift(-3)

In [9]:
main_dataframe.head()
main_dataframe.values

array([[8.71719971e+02, 5.67536100e+00, 6.48954980e+03, ...,
        9.65800020e+01, 9.64720000e+00, 9.65000000e+01],
       [8.70859985e+02, 2.68565770e+01, 6.48737988e+03, ...,
        9.66600040e+01, 3.14387024e+02, 9.63899990e+01],
       [8.70099976e+02, 1.12430000e+00, 6.47941016e+03, ...,
        9.65700000e+01, 7.71297990e+01, 9.65199970e+01],
       ...,
       [5.31479980e+02, 1.69000000e-02, 6.71500000e+03, ...,
        5.80200000e+01, 6.95349700e+00,            nan],
       [5.31479980e+02, 2.99520000e-01, 6.71500000e+03, ...,
        5.80800020e+01, 2.02403183e+02,            nan],
       [5.31630005e+02, 3.52891300e+00, 6.71500000e+03, ...,
        5.80900000e+01, 1.60602554e+02,            nan]])

In [10]:
def setValue(x , y):
    if y>x:
        
        return 1
    elif x>y:
        return 0
main_dataframe['target'] = list(map(setValue , main_dataframe['LTC-USD_close'] , main_dataframe['feature']))

In [11]:
main_dataframe

,BCH-USD_close,BCH-USD_volume,BTC-USD_close,BTC-USD_volume,ETH-USD_close,ETH-USD_volume,LTC-USD_close,LTC-USD_volume,feature,target
time,,,,,,,,,,
1528968660,871.719971,5.675361,6489.549805,0.587100,486.010010,26.019083,96.580002,9.647200,96.500000,0.0
1528968720,870.859985,26.856577,6487.379883,7.706374,486.010010,26.019083,96.660004,314.387024,96.389999,0.0
1528968780,870.099976,1.124300,6479.410156,3.088252,486.000000,8.449400,96.570000,77.129799,96.519997,0.0
1528968840,870.789978,1.749862,6479.410156,1.404100,485.750000,26.994646,96.500000,7.216067,96.440002,0.0
1528968900,870.000000,1.680500,6479.979980,0.753000,486.000000,77.355759,96.389999,524.539978,96.470001,1.0
...,...,...,...,...,...,...,...,...,...,...
1535215020,531.479980,0.016868,6714.520020,1.002652,279.359985,8.790519,58.009998,7.301921,58.080002,1.0
1535215080,531.469971,0.013854,6714.520020,1.021925,279.369995,1.311763,58.020000,23.802017,58.090000,1.0
1535215140,531.479980,0.016900,6715.000000,3.645508,279.660004,11.752819,58.020000,6.953497,NaN,NaN


In [12]:
times = main_dataframe.index.values

Criterion = sorted(main_dataframe.index.values)[-int(0.1*len(times))]
train_time = main_dataframe[main_dataframe.index <= Criterion]
test_time = main_dataframe[main_dataframe.index > Criterion]


In [82]:
def preprocess_main_dataframe(df):
    df.drop('feature' , axis=1)
    for col in df.columns:
        if col != 'target':
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values)
    df.dropna(inplace=True)
    sequences = []
    pre_days = deque(maxlen=30)
    for i in df.values:
        pre_days.append([n for n in i[:-1]])
        if len(pre_days) == 30:
            sequences.append([np.array(pre_days) , i[-1]])
    shuffle(sequences)
    buys = []
    sells=[]
    for seq , target in sequences:
        if target == 0:
            sells.append([seq,target])
        else:
            buys.append([seq , target])
    lower = min(len(buys) , len(sells))
    buys = buys[:lower]
    sells = sells[:lower]
    sequential_data = buys + sells
    x = []
    y = []
    for seq , target in sequential_data:
        x.append(seq)
        y.append(target)
    return np.array(x) , np.array(y)

In [83]:
x_train , y_train = preprocess_main_dataframe(train_time)

<ipython-input-82-d6b0c116d5c1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].pct_change()
<ipython-input-82-d6b0c116d5c1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
<ipython-input-82-d6b0c116d5c1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = preprocessing.sca

In [84]:
x_test , y_test = preprocess_main_dataframe(test_time)

<ipython-input-82-d6b0c116d5c1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].pct_change()
<ipython-input-82-d6b0c116d5c1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
<ipython-input-82-d6b0c116d5c1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = preprocessing.sca

In [97]:
model = Sequential()
model.add(LSTM(64))
model.add(Dense(32  , activation='relu'))
model.add(Dense(1 ,activation='sigmoid'))

In [100]:
model.compile( optimizer='adam', metrics=['accuracy'] , loss='binary_crossentropy')

In [103]:
model.fit(x_train , y_train , batch_size=300 , epochs=10 , validation_data=(x_test , y_test))

Epoch 1/10
 48/237 [=====>........................] - ETA: 9s - loss: 0.6881 - accuracy: 0.5177

KeyboardInterrupt: 